# Day 15: Chiton
https://adventofcode.com/2021/day/15

In [7]:
import os
import networkx as nx
from typing import Tuple

%load_ext blackcellmagic

In [8]:
# input data.
def test_input_location() -> str:
    return os.path.join("data/day_15", "test_input.txt")


def input_location() -> str:
    return os.path.join("data/day_15", "input.txt")


def read_input(input_file: str) -> list[list[int]]:
    cave: list[list[int]] = []
    with open(input_file) as f:
        for line in f:
            if line.rstrip():
                cave.append([int(x) for x in line.strip()])
    return cave


def read_input_part_2(input_file: str, factor_increase: int = 5) -> list[list[int]]:
    cave_orig = read_input(input_file)

    def new_value(value: int) -> int:
        return 1 if value + 1 > 9 else value + 1

    # fill out the width
    final_cave: list[list[int]] = []
    for ridx, orig_row in enumerate(cave_orig):
        new_row = orig_row.copy() * factor_increase
        for x in range(len(orig_row), len(new_row), 1):
            new_row[x] = new_value(new_row[x - len(orig_row)])
        final_cave.append(new_row)

    # fill out the height
    for x in range(len(final_cave), len(cave_orig) * factor_increase, 1):
        starting_idx = x - len(cave_orig)
        new_row = list(map(new_value, final_cave[starting_idx]))
        final_cave.append(new_row)

    return final_cave


def to_network(cave: list[list[int]]) -> nx.DiGraph:
    """
    Transforms a 2d array of ints into a directed graph with the cell "risks" entered as
    edge weights.
    """
    g = nx.DiGraph()
    # add nodes
    for ridx, row in enumerate(cave):
        for cidx, value in enumerate(row):
            g.add_node((ridx, cidx), weight=value)

    # add edges
    for ridx, row in enumerate(cave):
        for cidx, value in enumerate(row):
            if g.has_node((ridx - 1, cidx)):
                g.add_edge(
                    (ridx, cidx),
                    (ridx - 1, cidx),
                    weight=g.nodes[(ridx - 1, cidx)]["weight"],
                )
            if g.has_node((ridx + 1, cidx)):
                g.add_edge(
                    (ridx, cidx),
                    (ridx + 1, cidx),
                    weight=g.nodes[(ridx + 1, cidx)]["weight"],
                )
            if g.has_node((ridx, cidx - 1)):
                g.add_edge(
                    (ridx, cidx),
                    (ridx, cidx - 1),
                    weight=g.nodes[(ridx, cidx - 1)]["weight"],
                )
            if g.has_node((ridx, cidx + 1)):
                g.add_edge(
                    (ridx, cidx),
                    (ridx, cidx + 1),
                    weight=g.nodes[(ridx, cidx + 1)]["weight"],
                )

    return g


def bottom_right(cave: list[list[int]]) -> Tuple[int, int]:
    rows = len(cave)
    cols = max([len(row) for row in cave])
    return (rows - 1, cols - 1)

### Part 1
Find the shortest weighted path between top-left and bottom-right.

In [9]:
# Read in input data and make DiGraph
test_cave_raw = read_input(test_input_location())
G = to_network(test_cave_raw)

# Determine the starting and ending cells (top left and bottom right)
start = (0, 0)
end = bottom_right(test_cave_raw)

# test the algorithm on the test example.
assert (
    sum(
        [
            G.nodes[node]["weight"]
            for node in nx.dijkstra_path(G, start, end, weight="weight")
        ]
    )
    - G.nodes[(0, 0)]["weight"]
    == 40
)

# print the full path as list of node names which we made coordiantes (0,0) is top left.
# nx.dijkstra_path(G, (0,0), (9,9), weight='weight')

In [10]:
# Read in input data and make DiGraph
cave_raw = read_input(input_location())
G = to_network(cave_raw)

# Determine the starting and ending cells (top left and bottom right)
start = (0, 0)
end = bottom_right(cave_raw)

# print the sum of the weights.
print(
    sum(
        [
            G.nodes[node]["weight"]
            for node in nx.dijkstra_path(G, start, end, weight="weight")
        ]
    )
    - G.nodes[(0, 0)]["weight"]
)

### Part 2


In [11]:
test_cave_raw = read_input_part_2(test_input_location(), factor_increase=5)
assert len(test_cave_raw) == 50  # test copy is 5x longer
assert test_cave_raw[0][49] == 6  # test width completion
assert test_cave_raw[8][12] == 1  # test rollover
assert test_cave_raw[49][49] == 9  # test height completion

# Test the same path finder code on the larger input
G = to_network(test_cave_raw)
start = (0, 0)
end = bottom_right(test_cave_raw)
assert (
    sum(
        [
            G.nodes[node]["weight"]
            for node in nx.dijkstra_path(G, start, end, weight="weight")
        ]
    )
    - G.nodes[(0, 0)]["weight"]
    == 315
)

In [12]:
cave_raw = read_input_part_2(input_location(), factor_increase=5)
G = to_network(cave_raw)

# Determine the starting and ending cells (top left and bottom right)
start = (0, 0)
end = bottom_right(cave_raw)

# print the sum of the weights.
print(
    sum(
        [
            G.nodes[node]["weight"]
            for node in nx.dijkstra_path(G, start, end, weight="weight")
        ]
    )
    - G.nodes[(0, 0)]["weight"]
)